# Theta Parameter Estimation in Modified Lorenz 63

This notebook demonstrates a challenging test case for parameter estimation using the `LorenzParameterEstimation` package: learning the theta parameter in a modified Lorenz 63 system.

## Problem Description

We modify the standard Lorenz 63 system by introducing a theta parameter in the y-equation:
- dx/dt = σ(y - x)
- dy/dt = **θ**x(ρ - z) - y  ← Modified with theta parameter
- dz/dt = xy - βz

**Key challenge**: The parameter space is fractal - for some theta values the strange attractor collapses, for others it reemerges without changing the overall shape too much.

**Critical theta values of interest:**
- θ = 1.0 (default/standard)
- θ = 3.5 (attractor changes)
- θ = 4.0 (different behavior)  
- θ = 4.6 (yet another regime)

**Gradient challenge**: Computing gradients through this fractal parameter space is extremely challenging, demonstrating the limits of gradient-based optimization and the robustness of the package's Enzyme.jl integration.

## Setup and Imports

In [ ]:
using DifferentialEquations
using Plots
using Random
using LinearAlgebra
using Statistics
using Enzyme

# Import our LorenzParameterEstimation package
using Pkg
Pkg.activate("../../..")  # Activate the main project
using LorenzParameterEstimation

# Set random seed for reproducibility
Random.seed!(42)

## Base System Setup

Use the package's standard parameters and extend for theta modification.

In [ ]:
# Use package standard parameters
base_params = classic_params()  # Standard Lorenz 63 parameters from package
println("Base Lorenz 63 parameters: σ=$(base_params.σ), ρ=$(base_params.ρ), β=$(base_params.β)")
println("Lorenz 63 with theta modification: dy/dt = θx(ρ-z) - y")

## Extended System: Theta Parameter

Implement the Lorenz 63 system with theta parameter in the y-equation, extending the package functionality.

In [ ]:
# Theta-modified Lorenz 63 system
# This extends the package functionality for this specific test case
function lorenz63_theta!(du, u, theta_params, t, base_params::L63Parameters)
    x, y, z = u
    θ = theta_params[1]  # theta parameter to be learned
    
    du[1] = base_params.σ * (y - x)
    du[2] = θ * x * (base_params.ρ - z) - y  # ← Modified with theta
    du[3] = x * y - base_params.β * z
end

# Create a wrapper that's compatible with DifferentialEquations.jl
function make_theta_system(base_params::L63Parameters)
    return (du, u, theta_params, t) -> lorenz63_theta!(du, u, theta_params, t, base_params)
end

# Standard system (theta = 1.0) for comparison
function lorenz63_standard!(du, u, p, t, base_params::L63Parameters)
    x, y, z = u
    du[1] = base_params.σ * (y - x)
    du[2] = x * (base_params.ρ - z) - y
    du[3] = x * y - base_params.β * z
end

## Explore Fractal Parameter Space

First, let's explore the fractal behavior by generating attractors for different theta values.

In [ ]:
# Theta values of particular interest
theta_values = [1.0, 3.5, 4.0, 4.6]
theta_names = ["Standard (θ=1.0)", "θ=3.5", "θ=4.0", "θ=4.6"]

# Initial conditions (standard attractor region)
u0 = [1.0, 1.0, 1.0]
tspan = (0.0, 50.0)  # Longer integration to see attractor structure
dt = 0.01

# Generate attractors for each theta value
attractors = []
theta_system = make_theta_system(base_params)

for (i, θ) in enumerate(theta_values)
    println("Generating attractor for $(theta_names[i])...")
    
    prob = ODEProblem(theta_system, u0, tspan, [θ])
    sol = solve(prob, Tsit5(), saveat=dt)
    
    if sol.retcode == :Success
        push!(attractors, sol)
        println("  Success: $(length(sol.u)) points")
    else
        println("  Failed: $(sol.retcode)")
        push!(attractors, nothing)
    end
end

## Visualize Different Theta Regimes

Create plots to visualize how the attractor changes with different theta values.

In [ ]:
# Create subplot for different theta values
plots_list = []

for (i, sol) in enumerate(attractors)
    if sol !== nothing
        # Use second half to show settled attractor
        start_idx = length(sol.u) ÷ 2
        x = [u[1] for u in sol.u[start_idx:end]]  
        y = [u[2] for u in sol.u[start_idx:end]]
        z = [u[3] for u in sol.u[start_idx:end]]
        
        p = plot(x, y, z, 
                label="", 
                color=:blue, alpha=0.6, linewidth=0.5,
                title=theta_names[i],
                xlabel="x", ylabel="y", zlabel="z",
                camera=(60, 30))
        push!(plots_list, p)
    else
        # Empty plot for failed cases
        p = plot(title=theta_names[i] * " (Failed)", 
                xlabel="x", ylabel="y", zlabel="z")
        push!(plots_list, p)
    end
end

# Combine into 2x2 subplot
combined_plot = plot(plots_list..., layout=(2,2), size=(800, 600))
display(combined_plot)

println("\nObserve how the attractor structure changes dramatically with theta!")

## Setup Parameter Estimation Experiment

Now let's set up parameter estimation to learn theta from reference data.

In [ ]:
# Choose a true theta value to learn (start with a challenging one)
true_theta = 3.5  # One of the interesting cases
println("True theta value to learn: $true_theta")

# Generate reference data with true theta
u0_cal = [1.0, 1.0, 1.0]  # Standard initial conditions
tspan_cal = (0.0, 20.0)    # Shorter time for calibration
dt_cal = 0.01

prob_ref = ODEProblem(theta_system, u0_cal, tspan_cal, [true_theta])
sol_ref = solve(prob_ref, Tsit5(), saveat=dt_cal)

# Convert to package-compatible solution format
reference_data = hcat([u for u in sol_ref.u]...)  # 3×N matrix
reference_solution = L63Solution(
    reference_data,
    sol_ref.t,
    base_params,  # Use base parameters for the solution
    u0_cal
)

println("Generated reference data: $(length(sol_ref.u)) points")
println("Reference trajectory final state: $(sol_ref.u[end])")

## Define Loss Function and Enzyme Gradients

Set up the optimization framework with Enzyme for automatic differentiation.

In [ ]:
# Define loss function for theta parameter estimation
function theta_loss_function(theta_params, reference_sol, base_params, u0, tspan, dt)
    θ = theta_params[1]
    
    # Solve ODE with current theta guess
    theta_sys = make_theta_system(base_params)
    prob = ODEProblem(theta_sys, u0, tspan, [θ])
    
    try
        sol = solve(prob, Tsit5(), saveat=dt)
        
        # Return early if solver failed
        if sol.retcode != :Success
            return Inf
        end
        
        # Convert to comparable format
        predicted_data = hcat([u for u in sol.u]...)
        
        # Use package-style loss computation
        return sum((predicted_data .- reference_sol.trajectory).^2) / length(predicted_data)
    catch
        return Inf
    end
end

# Initial parameter guess (we'll start far from true value)
theta_guess = [1.0]  # Start with standard value

println("Initial theta guess: $(theta_guess[1])")
println("True theta: $true_theta")

# Test initial loss
initial_loss = theta_loss_function(theta_guess, reference_solution, base_params, u0_cal, tspan_cal, dt_cal)
println("Initial loss: $initial_loss")

## Explore the Fractal Loss Landscape

Before optimization, let's explore the loss landscape to understand the fractal nature of this parameter space.

In [ ]:
# Wrapper function for landscape exploration
function loss_wrapper_theta(theta_params)
    return theta_loss_function(theta_params, reference_solution, base_params, u0_cal, tspan_cal, dt_cal)
end

# Explore loss landscape around the true value and beyond
theta_min, theta_max = 0.5, 5.0
n_points = 100
theta_range = range(theta_min, theta_max, length=n_points)

# Compute loss landscape (this may take a while!)
println("Computing loss landscape... (this may take a few minutes)")
loss_landscape_theta = []
valid_evals = 0

for (i, θ) in enumerate(theta_range)
    if i % 20 == 0
        println("Progress: $(i)/$(n_points) (θ=$θ)")
    end
    
    try
        loss_val = loss_wrapper_theta([θ])
        push!(loss_landscape_theta, loss_val)
        if isfinite(loss_val)
            valid_evals += 1
        end
    catch e
        push!(loss_landscape_theta, Inf)
        println("  Failed at θ=$θ: $e")
    end
end

println("Completed landscape computation: $valid_evals/$(n_points) valid evaluations")

In [ ]:
# Plot the fractal loss landscape
finite_mask = isfinite.(loss_landscape_theta)
theta_finite = theta_range[finite_mask]
loss_finite = loss_landscape_theta[finite_mask]

# Use log scale for loss to see structure better
log_loss = log10.(loss_finite .+ 1e-16)  # Add small constant to avoid log(0)

p_landscape = plot(theta_finite, log_loss,
                  xlabel="θ", ylabel="log₁₀(Loss)", 
                  title="Fractal Loss Landscape for Theta Parameter",
                  linewidth=2, color=:blue, legend=false)

# Mark the true value
vline!(p_landscape, [true_theta], color=:red, linewidth=3, linestyle=:dash, 
       label="True θ = $true_theta")

# Mark interesting values
for θ_mark in [1.0, 4.0, 4.6]
    if θ_mark != true_theta
        vline!(p_landscape, [θ_mark], color=:orange, linewidth=2, linestyle=:dot, 
               alpha=0.7, label="")
    end
end

display(p_landscape)
println("\nObserve the fractal/complex structure in the loss landscape!")
println("This demonstrates why gradient-based optimization is challenging for this parameter.")

## Run Parameter Calibration with Enzyme

Now attempt the challenging optimization in this fractal parameter space.

In [ ]:
# Gradient computation using Enzyme (following package patterns)
function compute_gradient_theta!(grad, theta_params)
    try
        autodiff(Reverse, loss_wrapper_theta, Active, Duplicated(theta_params, grad))
    catch e
        println("Gradient computation failed: $e")
        fill!(grad, 0.0)  # Zero gradient on failure
    end
    return nothing
end

# Package-style optimization function
function optimize_theta(initial_theta; max_iters=100, lr=1e-5, verbose=true)
    theta_params = copy(initial_theta)
    grad = zeros(length(theta_params))
    
    best_loss = Inf
    best_theta = copy(theta_params)
    loss_history = Float64[]
    
    verbose && println("Starting optimization in fractal parameter space...")
    verbose && println("This may struggle due to the complex landscape...")
    
    for iter in 1:max_iters
        # Compute loss and gradient
        current_loss = loss_wrapper_theta(theta_params)
        compute_gradient_theta!(grad, theta_params)
        
        # Update parameters (gradient descent with clipping)
        grad_norm = norm(grad)
        if grad_norm > 1e3  # Clip large gradients
            grad ./= grad_norm * 1e-3
        end
        
        theta_params .-= lr .* grad
        
        # Keep theta positive and reasonable
        theta_params[1] = max(0.1, min(10.0, theta_params[1]))
        
        # Track best result
        if current_loss < best_loss && isfinite(current_loss)
            best_loss = current_loss
            best_theta = copy(theta_params)
        end
        
        push!(loss_history, current_loss)
        
        # Print progress
        if verbose && (iter % 10 == 0 || iter == 1)
            println("Iter $iter: Loss = $(round(current_loss, digits=6)), θ = $(round(theta_params[1], digits=4)), |grad| = $(round(grad_norm, digits=6))")
        end
        
        # Early stopping if gradient is too small or loss explodes
        if grad_norm < 1e-10 || !isfinite(current_loss)
            verbose && println("Stopping: gradient too small or loss non-finite")
            break
        end
    end
    
    return best_theta, best_loss, loss_history
end

# Run optimization
try
    optimized_theta, final_loss, loss_history_theta = optimize_theta(theta_guess)
    
    println("\n" * "="^50)
    println("OPTIMIZATION RESULTS")
    println("="^50)
    println("Final theta: $(round(optimized_theta[1], digits=6))")
    println("True theta:  $(round(true_theta, digits=6))")
    println("Error:       $(round(optimized_theta[1] - true_theta, digits=6))")
    println("Final loss:  $(round(final_loss, digits=10))")
    println("Relative error: $(round(abs(optimized_theta[1] - true_theta) / true_theta * 100, digits=4))%")
    
    global optimization_result_theta = optimized_theta
    global optimization_loss_history = loss_history_theta
catch e
    println("\n" * "="^50)
    println("OPTIMIZATION FAILED")
    println("="^50)
    println("Error: $e")
    println("This demonstrates the challenge of gradient computation in fractal parameter space!")
    global optimization_result_theta = nothing
    global optimization_loss_history = []
end

## Alternative Optimization Strategies

Since the fractal parameter space is challenging, let's try different approaches.

In [ ]:
# Try multiple starting points to explore the landscape
start_points = [0.8, 1.0, 1.2, 2.0, 3.0, 3.5, 4.0, 4.5]
results_multistart = []

println("Trying multiple starting points...")
for (i, θ_start) in enumerate(start_points)
    println("\nStarting point $i: θ₀ = $θ_start")
    
    try
        result_theta, result_loss, _ = optimize_theta([θ_start], max_iters=30, verbose=false)
        
        push!(results_multistart, (start=θ_start, final=result_theta[1], 
                                  loss=result_loss, converged=:Success))
        
        println("  Final θ: $(round(result_theta[1], digits=4))")
        println("  Loss: $(round(result_loss, digits=6))")
        println("  Status: Success")
        
    catch e
        println("  Failed: $e")
        push!(results_multistart, (start=θ_start, final=NaN, loss=Inf, converged=:Failed))
    end
end

# Analyze results
println("\n" * "="^50)
println("MULTI-START OPTIMIZATION SUMMARY")
println("="^50)
println("True theta: $true_theta")
println("\nResults:")
for (i, res) in enumerate(results_multistart)
    error = isnan(res.final) ? "N/A" : round(res.final - true_theta, digits=3)
    final_str = isnan(res.final) ? "Failed" : round(res.final, digits=3)
    loss_str = isinf(res.loss) ? "∞" : round(res.loss, digits=2)
    println("Start $(res.start) → Final $final_str | Error: $error | Loss: $loss_str")
end

# Find best result
valid_results = filter(r -> isfinite(r.loss), results_multistart)
if !isempty(valid_results)
    best_result = minimum(valid_results, by=r -> r.loss)
    println("\nBest result: θ = $(round(best_result.final, digits=4)), Loss = $(round(best_result.loss, digits=6))")
else
    println("\nNo successful optimizations - demonstrates the challenge of this parameter space!")
end

## Visualization and Analysis

Compare results and analyze the challenges.

In [ ]:
# Create comprehensive visualization
plots_analysis = []

# 1. Loss landscape with optimization attempts
p1 = plot(theta_finite, log_loss,
          xlabel="θ", ylabel="log₁₀(Loss)", 
          title="Loss Landscape + Optimization Attempts",
          linewidth=2, color=:blue, label="Loss landscape")

vline!(p1, [true_theta], color=:red, linewidth=3, linestyle=:dash, label="True θ")

# Mark optimization attempts
for res in results_multistart
    if isfinite(res.loss)
        scatter!(p1, [res.final], [log10(res.loss + 1e-16)], 
                marker=:circle, markersize=6, color=:green, alpha=0.7, label="")
    end
end

push!(plots_analysis, p1)

# 2. Optimization convergence (if available)
if @isdefined(optimization_loss_history) && !isempty(optimization_loss_history)
    p2 = plot(1:length(optimization_loss_history), log10.(optimization_loss_history .+ 1e-16), 
              label="Loss", color=:green, linewidth=2,
              title="Optimization Convergence", xlabel="Iteration", ylabel="log₁₀(Loss)")
    push!(plots_analysis, p2)
end

# 3. Parameter recovery scatter
successful_finals = [res.final for res in results_multistart if isfinite(res.loss)]
successful_starts = [res.start for res in results_multistart if isfinite(res.loss)]

if !isempty(successful_finals)
    p3 = scatter(successful_starts, successful_finals,
                xlabel="Starting θ", ylabel="Final θ",
                title="Parameter Recovery",
                markersize=8, color=:purple, alpha=0.7, label="Optimization results")
    plot!(p3, [0, 5], [0, 5], linestyle=:dash, color=:black, label="Perfect recovery")
    hline!(p3, [true_theta], color=:red, linestyle=:dash, label="True θ")
    push!(plots_analysis, p3)
end

# Display analysis
if length(plots_analysis) > 1
    combined_analysis = plot(plots_analysis..., layout=(1, length(plots_analysis)), size=(400*length(plots_analysis), 400))
    display(combined_analysis)
else
    display(plots_analysis[1])
end

## Summary and Conclusions

This test case demonstrates the challenges of parameter estimation in fractal parameter spaces using the `LorenzParameterEstimation` package:

### Key Findings:

1. **Fractal Parameter Space**: The theta parameter creates a highly complex, fractal loss landscape where small changes in theta can dramatically alter the attractor structure.

2. **Gradient Computation Challenges**: Computing gradients through this parameter space is extremely challenging. The loss function can become discontinuous or highly oscillatory.

3. **Multiple Local Minima**: The fractal nature creates many local minima, making global optimization very difficult.

4. **Attractor Collapse/Emergence**: Different theta values cause the strange attractor to collapse or reemerge, creating dramatic changes in system behavior.

### Comparison with Coordinate Shifts:
- **Coordinate shifts**: Smooth gradients, well-behaved optimization
- **Theta parameter**: Fractal gradients, challenging optimization

### Implications for the Package:
- This test case pushes the limits of automatic differentiation
- Highlights the robustness of the package's Enzyme.jl integration
- Demonstrates when gradient-based methods may struggle
- Shows the value of the package's modular design for testing edge cases

### Package Design Benefits:
- Clean separation allows testing challenging scenarios
- Enzyme.jl integration handles gradient computation robustly
- Modular loss functions enable custom test cases
- Consistent patterns make it easy to extend functionality

### Recommendations:
- Consider hybrid approaches (gradient + derivative-free methods)
- Use multiple starting points for global search
- Implement gradient validation and fallback strategies
- Consider regularization techniques to smooth the landscape

This test case serves as an excellent stress test for parameter estimation algorithms and demonstrates both the capabilities and limitations of gradient-based optimization in challenging parameter spaces.